In [1]:
movies=sc.textFile('ml-100k/u.item')

In [2]:
movies.first()

u'1|Toy Story (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Toy%20Story%20(1995)|0|0|0|1|1|1|0|0|0|0|0|0|0|0|0|0|0|0|0'

In [5]:
genres=sc.textFile('ml-100k/u.genre')

In [6]:
genres.take(5)

[u'unknown|0', u'Action|1', u'Adventure|2', u'Animation|3', u"Children's|4"]

In [11]:
genre_map=genres.filter(lambda x:len(x)>0).map(lambda line:line.split('|')).map(lambda x:(x[1],x[0])).collectAsMap()

In [13]:
genre_map

{u'0': u'unknown',
 u'1': u'Action',
 u'10': u'Film-Noir',
 u'11': u'Horror',
 u'12': u'Musical',
 u'13': u'Mystery',
 u'14': u'Romance',
 u'15': u'Sci-Fi',
 u'16': u'Thriller',
 u'17': u'War',
 u'18': u'Western',
 u'2': u'Adventure',
 u'3': u'Animation',
 u'4': u"Children's",
 u'5': u'Comedy',
 u'6': u'Crime',
 u'7': u'Documentary',
 u'8': u'Drama',
 u'9': u'Fantasy'}

In [16]:
movies_title=movies.map(lambda x:x.split('|')).map(lambda x:x[1])

In [21]:
#查看电影的题材, 0表示不属于该题材, 1表示属于该题材
movies_genre=movies.map(lambda x:x.split('|')).map(lambda x:x[5:])

In [27]:
def func(rdd):
    genres=rdd[5:]
    genres_assigned=zip(genres,range(len(genres)))
    index=[]
    for genre,idx in genres_assigned:
        if genre=='1':
            index.append(idx)
    index_val=[genre_map[str(i)] for i in index]#根据编码字典找出索引的相应题材名
    index_val_str=','.join(index_val)
    return (int(rdd[0]),rdd[1]+','+index_val_str)

In [28]:
titles_and_genres=movies.map(lambda x:x.split('|')).map(lambda x:func(x))

In [29]:
titles_and_genres.take(5)

[(1, u"Toy Story (1995),Animation,Children's,Comedy"),
 (2, u'GoldenEye (1995),Action,Adventure,Thriller'),
 (3, u'Four Rooms (1995),Thriller'),
 (4, u'Get Shorty (1995),Action,Comedy,Drama'),
 (5, u'Copycat (1995),Crime,Drama,Thriller')]

In [30]:
from pyspark.mllib.recommendation import ALS
from pyspark.mllib.recommendation import Rating

In [31]:
raw_data=sc.textFile('ml-100k/u.data')

In [32]:
raw_ratings=raw_data.map(lambda x:x.split('\t')[:3])

In [34]:
ratings=raw_ratings.map(lambda x:Rating(x[0],x[1],x[2]))

In [37]:
ratings.first()

Rating(user=196, product=242, rating=3.0)

In [36]:
ratings.cache()

PythonRDD[22] at RDD at PythonRDD.scala:48

In [38]:
als_model=ALS.train(ratings,50,5,0.1)

In [39]:
from pyspark.mllib.linalg import Vectors

In [40]:
als_model.productFeatures().first()

(4,
 array('d', [0.2329028844833374, 0.08546625822782516, -0.5436590313911438, 0.16188177466392517, 0.23754294216632843, -0.28258490562438965, -0.37231311202049255, 0.15786513686180115, -0.07673388719558716, 0.14272460341453552, 0.14980344474315643, 0.046577535569667816, 0.12659142911434174, 0.310492604970932, -0.03853148967027664, -0.384851336479187, -0.27536699175834656, 0.10113581269979477, -0.34300515055656433, 0.1455269455909729, 0.0919487476348877, 0.035582996904850006, -0.12395696341991425, -0.08984935283660889, -0.5159639716148376, 0.13730713725090027, -0.3919117748737335, 0.39358022809028625, -0.37092119455337524, -0.17356930673122406, -0.4846462309360504, -0.4105401635169983, 0.38906723260879517, 0.21525654196739197, -0.07056694477796555, -0.035239528864622116, 0.12631778419017792, -0.047907982021570206, -0.2267122119665146, 0.21900178492069244, 0.08328519761562347, -0.20447097718715668, -0.06795285642147064, -0.14225022494792938, 0.03823564574122429, -0.36189401149749756, 0.

In [83]:
movie_factors=als_model.productFeatures().map(lambda (id,factor):(id,Vectors.dense(factor)))

In [84]:
movie_vectors=movie_factors.map(lambda (id,vec):vec)

In [85]:
user_factors=als_model.userFeatures().map(lambda (id,factor):(id,Vectors.dense(factor)))

In [86]:
user_vectors=user_factors.map(lambda (id,vec):vec)

In [87]:
from pyspark.mllib.linalg.distributed import RowMatrix

In [88]:
movie_matrix=RowMatrix(movie_vectors)

In [89]:
user_matrix=RowMatrix(user_vectors)

In [90]:
from pyspark.mllib.stat import MultivariateStatisticalSummary

In [91]:
desc_moive_matrix=MultivariateStatisticalSummary(movie_matrix.rows)

In [92]:
desc_user_matrix=MultivariateStatisticalSummary(user_matrix.rows)

Exception AttributeError: "'RDD' object has no attribute '_detach'" in <bound method MultivariateStatisticalSummary.__del__ of <pyspark.mllib.stat._statistics.MultivariateStatisticalSummary object at 0x1057f8690>> ignored


In [93]:
desc_moive_matrix.mean()

array([ 0.12522839,  0.10677686, -0.38345904,  0.17202698,  0.18109234,
       -0.1972895 , -0.26634585,  0.03048677, -0.0884068 ,  0.05189915,
        0.04266586,  0.02307294,  0.10639708, -0.03525954,  0.11365839,
       -0.32185592, -0.01177112, -0.00940092, -0.07285093,  0.08175648,
        0.225861  ,  0.00818854, -0.11863219,  0.00597933, -0.35364753,
        0.05647397, -0.2171885 ,  0.50533588, -0.26315251, -0.17478401,
       -0.35462753, -0.10779531,  0.28178213,  0.09800288,  0.09824397,
       -0.01158634,  0.14504087,  0.00171914, -0.19554569,  0.09658699,
        0.33829668, -0.16422472,  0.00691622, -0.04394762, -0.03306095,
       -0.27331759,  0.16573033, -0.0902899 ,  0.03666803, -0.05987156])

In [94]:
desc_user_matrix.mean()

array([ 0.18320141,  0.14863872, -0.68754204,  0.24085708,  0.34136779,
       -0.33344052, -0.43088745,  0.08013158, -0.17137994,  0.09652387,
        0.01047763,  0.06452675,  0.18193983, -0.00801536,  0.16326213,
       -0.52296946, -0.00957119,  0.03058327, -0.17350703,  0.13522944,
        0.39933366, -0.01212082, -0.18200787,  0.05961164, -0.61193131,
        0.09890991, -0.34447208,  0.84677932, -0.39685447, -0.34289227,
       -0.61395307, -0.19593843,  0.49625961,  0.23762673,  0.17104971,
       -0.02189424,  0.2277889 ,  0.01372809, -0.33755487,  0.16508937,
        0.55975714, -0.27422564,  0.02200518, -0.11273535, -0.05752504,
       -0.38277838,  0.25702858, -0.10577996,  0.05931806, -0.07647809])

In [73]:
from pyspark.mllib.clustering import KMeans

In [74]:
num_clusters=5
num_iterations=20
num_runs=3

In [95]:
movie_cluster_model=KMeans.train(movie_vectors,num_clusters,num_iterations,num_runs)

/Users/qinkevin/spark/python/pyspark/mllib/clustering.py:347: UserWarning: The param `runs` has no effect since Spark 2.0.0.
  warnings.warn("The param `runs` has no effect since Spark 2.0.0.")


In [96]:
predictions=movie_cluster_model.predict(movie_vectors)

In [97]:
predictions.take(10)

[3, 4, 3, 3, 4, 3, 0, 3, 2, 2]

In [98]:
titles_factors=titles_and_genres.join(movie_factors)

In [106]:
titles_factors.first()

(1536,
 (u'Aiqing wansui (1994),Drama',
  DenseVector([0.0427, 0.3265, -0.5593, 0.3783, 0.1329, -0.0512, -0.1642, -0.0569, -0.0085, 0.1177, 0.2506, 0.0663, 0.1504, -0.2226, 0.0521, -0.7047, -0.4408, -0.161, -0.2365, 0.0107, 0.196, -0.1977, -0.3612, -0.1332, -0.3551, -0.1072, -0.3198, 0.759, -0.543, 0.0882, -0.5283, -0.2813, 0.1971, 0.1312, -0.1702, 0.3334, 0.1662, -0.2437, 0.021, -0.2707, 0.1206, -0.3992, -0.0615, 0.09, -0.3206, -0.0598, 0.2428, -0.0071, -0.0304, -0.1719])))

In [121]:
def func2(rdd):
    id,(name_genres,vec)=rdd
    
    pred=movie_cluster_model.predict(vec)
    cluster_center=movie_cluster_model.clusterCenters[pred]
    cluster_center_vec=Vectors.dense(cluster_center)
    dist=vec.squared_distance(cluster_center_vec)
    return u'电影' + str(id) + u'的题材类型是' + name_genres + ',' + u'聚类模型预测的标签是' + str(pred)+ ',' + \
           u'与聚类所属类别中心的距离是' + str(dist)

In [122]:
movies_assigned=titles_factors.map(lambda x:func2(x))

In [124]:
for i in movies_assigned.take(5):
    print i

电影1536的题材类型是Aiqing wansui (1994),Drama,聚类模型预测的标签是4,与聚类所属类别中心的距离是1.2790866661
电影1026的题材类型是Lay of the Land, The (1997),Comedy,Drama,聚类模型预测的标签是2,与聚类所属类别中心的距离是1.43238489242
电影516的题材类型是Local Hero (1983),Comedy,聚类模型预测的标签是4,与聚类所属类别中心的距离是1.54784857014
电影6的题材类型是Shanghai Triad (Yao a yao yao dao waipo qiao) (1995),Drama,聚类模型预测的标签是4,与聚类所属类别中心的距离是2.24922266919
电影1032的题材类型是Little Big League (1994),Children's,Comedy,聚类模型预测的标签是2,与聚类所属类别中心的距离是1.14068859189


In [125]:
movie_cost=movie_cluster_model.computeCost(movie_vectors)

In [126]:
movie_cost

2157.689569574677

In [127]:
train_test_split_movies=movie_vectors.randomSplit([0.6,0.4],123)

In [128]:
train_movies=train_test_split_movies[0]

In [129]:
test_movies=train_test_split_movies[1]

In [132]:
for k in [2,3,4,5,10,20]:
    k_model=KMeans.train(train_movies,num_iterations,k,num_runs)
    cost=k_model.computeCost(test_movies)
    print (k,cost)

(2, 778.4055494370384)
(3, 777.2776268988299)
(4, 768.0468228245023)
(5, 775.7348702912013)
(10, 760.734045649849)
(20, 774.0467673294866)
